In [346]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [347]:
def get_list_of_university_towns():
    #'''Returns a DataFrame of towns and the states they are in from the 
    #university_towns.txt list. The format of the DataFrame should be:
    #DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    #columns=["State", "RegionName"]  )
        
    
    #The following cleaning needs to be done:

    #1. For "State", removing characters from "[" to the end.
    #2. For "RegionName", when applicable, removing every character from " (" to the end.
    #3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    data = pd.read_csv('university_towns.txt', sep=r'^.+\["edit"\]' , header=None, engine='python')
    data.columns = ["RegionName"]
        
    data['State'] = data['RegionName'].where(data['RegionName'].str.endswith('[edit]'))
        
    data['State'] = data['State'].fillna(method='ffill')
         
    data['State'] = data['State'].str.replace('\[.*\]','')
    data['RegionName'] = data['RegionName'].str.replace('\[.*\]','')
    data['RegionName'] = data['RegionName'].str.replace('\(.*\)','')
    
    
    data.drop(data[data['RegionName'] == data['State']].index,inplace=True)
    data.reset_index(inplace=True,drop=True)
    
    data['State'] = data['State'].str.strip()
    data['RegionName'] = data['RegionName'].str.strip()
    cols = ['State','RegionName']
    data = data[cols]
    
    data.set_index('State',inplace=True)
    #data.to_records().tolist()
    
    return data

In [348]:
def convert_housing_data_to_quarters():
    data = pd.read_csv('City_Zhvi_AllHomes.csv')
    data.replace({"State": states},inplace=True)
    data.set_index(['State','RegionName'],drop=True, inplace=True)
    
    for i in range(1997,2000):
        for j in range(1,13):
            if ( j < 10):
                j = '0' + str(j)
            data.drop(str(i) +'-'+ str(j),axis=1,inplace=True)
            
    for j in range(4,13):
        if ( j < 10):
                j = '0' + str(j)
        data.drop('1996' +'-'+ str(j),axis=1,inplace=True)

    for i in range(2000,2016):
        data[str(i)+"q1"] = data[[str(i)+'-'+str('01'),str(i)+'-'+str('02'),str(i)+'-'+str('03')]].mean(axis=1)
        data[str(i)+"q2"] = data[[str(i)+'-'+str('04'),str(i)+'-'+str('05'),str(i)+'-'+str('06')]].mean(axis=1)
        data[str(i)+"q3"] = data[[str(i)+'-'+str('07'),str(i)+'-'+str('08'),str(i)+'-'+str('09')]].mean(axis=1)
        data[str(i)+"q4"] = data[[str(i)+'-'+str('10'),str(i)+'-'+str('11'),str(i)+'-'+str('12')]].mean(axis=1)
        
    data[str(2016)+"q1"] = data[[str(2016)+'-'+str('01'),str(2016)+'-'+str('02'),str(2016)+'-'+str('03')]].mean()   
    data[str(2016)+"q2"] = data[[str(2016)+'-'+str('04'),str(2016)+'-'+str('05'),str(2016)+'-'+str('06')]].mean()   
    data[str(2016)+"q3"] = data[[str(2016)+'-'+str('07'),str(2016)+'-'+str('08')]].mean()   
    
    for i in range(2000,2016):
        for j in range(1,13):
            if ( j < 10):
                j = '0' + str(j)
            data.drop(str(i) +'-'+ str(j),axis=1,inplace=True)
            
    for j in range(1,9):
        if ( j < 10):
            j = '0' + str(j)
        data.drop(str(2016) +'-'+ str(j),axis=1,inplace=True)   
    data.drop(['RegionID','Metro','CountyName','SizeRank'],axis=1,inplace=True)
    return data

In [349]:
    import pandas as pd
    import numpy as np
    gdp_data = pd.ExcelFile('gdplev.xls')
    gdp_data.sheet_names
    gdp = gdp_data.parse("Sheet1")
    gdp = gdp.dropna(axis=1,how='all')
    gdp = gdp.dropna(axis=0,how='all')    
    gdp.columns = ['Period-A','GDP1-A','GDP2-A','Quarter','GDP1-Q','GDP chained']
    
    gdp.drop(['Period-A','GDP1-A','GDP2-A','GDP1-Q'],axis=1,inplace=True)
    gdp.drop(gdp.index[:3], inplace=True)
    gdp.reset_index(inplace=True,drop=True)
    
    gdp['GDP Change'] = gdp['GDP chained'] - gdp['GDP chained'].shift(1)
    
    gdp['Change'] = gdp['GDP Change'].apply(lambda x: "decline" if (x < 0 ) else "growth" )
    
    recession_start = gdp['Quarter'].where((gdp['Change'] == "decline") & (gdp['Change'].shift(-1) == "decline") & (gdp['Change'].shift(1) == "growth") & gdp['Quarter'].str.startswith("2"))
    
    recession_end = gdp['Quarter'].where((gdp['Change'] == "growth") & (gdp['Change'].shift(1) == "growth") & (gdp['Change'].shift(2) == "decline") & (gdp['Change'].shift(3) == "decline") & gdp['Quarter'].str.startswith("2"))
    
  
    
    rstart = recession_start[~recession_start.isnull()]
    rend = recession_end[~recession_end.isnull()]
    
    housing_data = convert_housing_data_to_quarters()
    #hp_recession = housing_data.loc[:,'2000q1':'2000q4']
    #hp_recession = housing_data[str(rstart.iloc[0]):str(rend.iloc[0])]
    #hp_recession
    
    #gdp.set_index(['Quarter'],inplace=True)
    #res_gdp = gdp[str(rstart.iloc[0]):str(rend.iloc[0])]
    
    #s = str(res_gdp.loc[res_gdp['GDP chained'] == res_gdp['GDP chained'].min()].index.values[0])
    #res_gdp

In [432]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    gdp_data = pd.ExcelFile('gdplev.xls')
    gdp_data.sheet_names
    gdp = gdp_data.parse("Sheet1")
    gdp = gdp.dropna(axis=1,how='all')
    gdp = gdp.dropna(axis=0,how='all')    
    gdp.columns = ['Period-A','GDP1-A','GDP2-A','Quarter','GDP1-Q','GDP chained']
    
    gdp.drop(['Period-A','GDP1-A','GDP2-A','GDP1-Q'],axis=1,inplace=True)
    gdp.drop(gdp.index[:3], inplace=True)
    gdp.reset_index(inplace=True,drop=True)
    
    gdp['GDP Change'] = gdp['GDP chained'] - gdp['GDP chained'].shift(1)
    
    gdp['Change'] = gdp['GDP Change'].apply(lambda x: "decline" if (x < 0 ) else "growth" )
    
    recession_start = gdp['Quarter'].where((gdp['Change'] == "decline") & (gdp['Change'].shift(-1) == "decline") & (gdp['Change'].shift(1) == "growth") & gdp['Quarter'].str.startswith("2"))
    
    recession_end = gdp['Quarter'].where((gdp['Change'] == "growth") & (gdp['Change'].shift(1) == "growth") & (gdp['Change'].shift(2) == "decline") & (gdp['Change'].shift(3) == "decline") & gdp['Quarter'].str.startswith("2"))
    
  
    
    rstart = recession_start[~recession_start.isnull()]
    rend = recession_end[~recession_end.isnull()]
    
    
    
    gdp.set_index(['Quarter'],inplace=True)
    res_gdp = gdp[str(rstart.iloc[0]):str(rend.iloc[0])]
    
    #s = str(res_gdp.loc[res_gdp['GDP chained'] == res_gdp['GDP chained'].min()].index.values[0])
    
    return res_gdp

In [433]:
recession_data = get_recession_bottom()
s = recession_data.index.values

housing_data = convert_housing_data_to_quarters()
hp_recession = housing_data.loc[:,str(s[0]):str(s[len(s) - 1])]

hp_recession.head()
#univ_town = get_list_of_university_towns()
#univ_town.head()


,,2008q3,2008q4,2009q1,2009q2,2009q3,2009q4
State,RegionName,,,,,,
New York,New York,499766.666667,487933.333333,477733.333333,465833.333333,455933.333333,458366.666667
California,Los Angeles,469500.000000,443966.666667,426266.666667,413900.000000,406366.666667,404333.333333
Illinois,Chicago,232000.000000,227033.333333,223766.666667,219700.000000,214100.000000,211666.666667
Pennsylvania,Philadelphia,116933.333333,115866.666667,116200.000000,116166.666667,116733.333333,118566.666667
Arizona,Phoenix,193766.666667,183333.333333,177566.666667,168233.333333,155933.333333,143466.666667


In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"

In [365]:
univ_town = get_list_of_university_towns()
univ_town.reset_index(inplace=True)
univ_town.set_index(['State','RegionName'],drop=True,inplace=True)

univ_town.head()

Empty DataFrame
Columns: []
Index: [(Alabama, Auburn), (Alabama, Florence), (Alabama, Jacksonville), (Alabama, Livingston), (Alabama, Montevallo)]

In [391]:
housing_data = convert_housing_data_to_quarters()
hp_recession = housing_data.loc[:,str(s[0]):str(s[len(s) - 1])]
hp_recession.head()

,,2008q3,2008q4,2009q1,2009q2,2009q3,2009q4
State,RegionName,,,,,,
New York,New York,499766.666667,487933.333333,477733.333333,465833.333333,455933.333333,458366.666667
California,Los Angeles,469500.000000,443966.666667,426266.666667,413900.000000,406366.666667,404333.333333
Illinois,Chicago,232000.000000,227033.333333,223766.666667,219700.000000,214100.000000,211666.666667
Pennsylvania,Philadelphia,116933.333333,115866.666667,116200.000000,116166.666667,116733.333333,118566.666667
Arizona,Phoenix,193766.666667,183333.333333,177566.666667,168233.333333,155933.333333,143466.666667


In [393]:
# inner join
univ_town_hp_df = pd.merge(hp_recession, univ_town, how='inner', left_index=True, right_index=True)
univ_town_hp_df.head()

2008q3         2008q4         2009q1  \
State   RegionName                                                
Alabama Montevallo  127266.666667  125800.000000  124033.333333   
        Tuscaloosa  139600.000000  140100.000000  139133.333333   
Alaska  Fairbanks   249966.666667  242900.000000  234966.666667   
Arizona Flagstaff   322633.333333  318733.333333  309400.000000   
        Tempe       228133.333333  219766.666667  214666.666667   

                           2009q2         2009q3         2009q4  
State   RegionName                                               
Alabama Montevallo  125200.000000  122666.666667  124133.333333  
        Tuscaloosa  136933.333333  137233.333333  136533.333333  
Alaska  Fairbanks   225833.333333  225766.666667  224033.333333  
Arizona Flagstaff   299600.000000  290900.000000  284000.000000  
        Tempe       207500.000000  196300.000000  189133.333333

In [401]:
non_univ_town_hp = pd.merge(hp_recession,univ_town, how='left', left_index=True, right_index=True,indicator=True)
non_univ_town_hp_df = non_univ_town_hp.loc[non_univ_town_hp['_merge'] == "left_only"]
non_univ_town_hp_df.drop(['_merge'],axis=1,inplace=True)
non_univ_town_hp_df.head()

C:\Users\Gracy\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2008q3         2008q4         2009q1  \
State   RegionName                                                 
Alabama Adamsville    84866.666667   85833.333333   85433.333333   
        Alabaster    155333.333333  153200.000000  150533.333333   
        Albertville   85333.333333   83766.666667   84200.000000   
        Arab         108933.333333  107933.333333  113600.000000   
        Ardmore      149100.000000  151200.000000  147133.333333   

                            2009q2         2009q3         2009q4  
State   RegionName                                                
Alabama Adamsville    85766.666667   84766.666667   82933.333333  
        Alabaster    149666.666667  149600.000000  149266.666667  
        Albertville   84333.333333   84666.666667   86900.000000  
        Arab         115366.666667  113533.333333  110733.333333  
        Ardmore      143900.000000  142800.000000  142466.666667

In [ ]:
'''Merge method	SQL Join Name	Description
left	LEFT OUTER JOIN	Use keys from left frame only
right	RIGHT OUTER JOIN	Use keys from right frame only
outer	FULL OUTER JOIN	Use union of keys from both frames
inner	INNER JOIN	Use intersection of keys from both frames'''

In [406]:
#2-sample t-test: testing for difference across populations
'''Notes
-----
We can use this test, if we observe two independent samples from
the same or different population, e.g. exam scores of boys and
girls or of two ethnic groups. The test measures whether the
average (expected) value differs significantly across samples. If
we observe a large p-value, for example larger than 0.05 or 0.1,
then we cannot reject the null hypothesis of identical average scores.
If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%,
then we reject the null hypothesis of equal averages.
''' 
'''nan_policy : {'propagate', 'raise', 'omit'}, optional
    Defines how to handle when input contains nan. 'propagate' returns nan,
    'raise' throws an error, 'omit' performs the calculations ignoring nan
    values. Default is 'propagate'.
'''

ttest_ind(univ_town_hp_df,non_univ_town_hp_df,nan_policy='omit')


Ttest_indResult(statistic=masked_array(data = [-2.4631142199028044 -2.3937467872237588 -2.355273790203779
 -2.3010534410950516 -2.151183364656689 -2.1051135236180762],
             mask = [False False False False False False],
       fill_value = 1e+20)
, pvalue=masked_array(data = [ 0.01379052  0.01669579  0.01852844  0.02140923  0.03148543  0.03530609],
             mask = False,
       fill_value = 1e+20)
)

In [404]:
#(different, p, better)
(True,[ 0.01379052  0.01669579  0.01852844  0.02140923  0.03148543  0.03530609],)

In [ ]:
#p value is less than 0.05 or 0.1 ..there
#fore we reject the null hypothesis..which is during recession the hp in university town values not better than the non-university towns values
#at 95% confidence level 
#p-value is greater than the corresponding significance level of 5%. accept null hyp (  pvalue=0.090731043439577483)

In [ ]:
''' Run a t-test to compare the ratio of the mean price of houses in university towns
the quarter before the recession starts compared to the recession bottom.
(price_ratio=quarter_before_recession/recession_bottom)'''
'''The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).''''''
    


In [410]:

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    gdp_data = pd.ExcelFile('gdplev.xls')
    gdp_data.sheet_names
    gdp = gdp_data.parse("Sheet1")
    gdp = gdp.dropna(axis=1,how='all')
    gdp = gdp.dropna(axis=0,how='all')    
    gdp.columns = ['Period-A','GDP1-A','GDP2-A','Quarter','GDP1-Q','GDP chained']
    
    gdp.drop(['Period-A','GDP1-A','GDP2-A','GDP1-Q'],axis=1,inplace=True)
    gdp.drop(gdp.index[:3], inplace=True)
    gdp.reset_index(inplace=True,drop=True)
    
    gdp['GDP Change'] = gdp['GDP chained'] - gdp['GDP chained'].shift(1)
    
    gdp['Change'] = gdp['GDP Change'].apply(lambda x: "decline" if (x < 0 ) else "growth" )
    
    s = gdp['Quarter'].where((gdp['Change'] == "decline") & (gdp['Change'].shift(-1) == "decline") & (gdp['Change'].shift(1) == "growth") & gdp['Quarter'].str.startswith("2"))
    
    res = s[~s.isnull()].values[0]
    return res

In [408]:

def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    gdp_data = pd.ExcelFile('gdplev.xls')
    gdp_data.sheet_names
    gdp = gdp_data.parse("Sheet1")
    gdp = gdp.dropna(axis=1,how='all')
    gdp = gdp.dropna(axis=0,how='all')    
    gdp.columns = ['Period-A','GDP1-A','GDP2-A','Quarter','GDP1-Q','GDP chained']
    

    gdp.drop(['Period-A','GDP1-A','GDP2-A','GDP1-Q'],axis=1,inplace=True)
    gdp.drop(gdp.index[:3], inplace=True)
    gdp.reset_index(inplace=True,drop=True)
    
    gdp['GDP Change'] = gdp['GDP chained'] - gdp['GDP chained'].shift(1)
    
    gdp['Change'] = gdp['GDP Change'].apply(lambda x: "decline" if (x < 0 ) else "growth" )
    
    recession_start = gdp['Quarter'].where((gdp['Change'] == "decline") & (gdp['Change'].shift(-1) == "decline") & (gdp['Change'].shift(1) == "growth") & gdp['Quarter'].str.startswith("2"))
    
    recession_end = gdp['Quarter'].where((gdp['Change'] == "growth") & (gdp['Change'].shift(1) == "growth") & (gdp['Change'].shift(2) == "decline") & (gdp['Change'].shift(3) == "decline") & gdp['Quarter'].str.startswith("2"))
    
  
    
    rstart = recession_start[~recession_start.isnull()]
    rend = recession_end[~recession_end.isnull()]
    
    
    
    gdp.set_index(['Quarter'],inplace=True)
    res_gdp = gdp[str(rstart.iloc[0]):str(rend.iloc[0])]
    
    s = str(res_gdp.loc[res_gdp['GDP chained'] == res_gdp['GDP chained'].min()].index.values[0])
    
    return s

In [448]:

#df['2008q2']


In [412]:
rbottom = get_recession_bottom()
rbottom

'2009q2'

In [430]:
rbottom = get_recession_bottom()
df = convert_housing_data_to_quarters()
rstart = get_recession_start()
df['price ratio'] = df.iloc[:,df.columns.get_loc(rstart) - 1] / df[rbottom]
df.head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3,price ratio
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,NaN,NaN,NaN,1.081789
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,NaN,NaN,NaN,1.213498
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,NaN,NaN,NaN,1.082840
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,NaN,NaN,NaN,1.016930
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,NaN,NaN,NaN,1.220131
